In [ ]:
import tensorflow as tf
import os
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.applications import MobileNetV3Small, ResNet50,  ConvNeXtTiny, Xception, DenseNet121, VGG19, InceptionV3, InceptionResNetV2, NASNetMobile
import matplotlib.pyplot as plt
import zipfile

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

In [ ]:
zip_path = '/content/drive/MyDrive/original+smote+image_processing.zip'

extracted_path = '/content/extracted_images/'

os.makedirs(extracted_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

extracted_folders = os.listdir(extracted_path)
print("Extracted folders:", extracted_folders)

for folder in extracted_folders:
    folder_path = os.path.join(extracted_path, folder)
    if os.path.isdir(folder_path):
        images_in_folder = os.listdir(folder_path)
        print(f"Images in '{folder}':", images_in_folder)

In [ ]:
train_dir = '/content/extracted_images/split/train'
test_dir = '/content/extracted_images/split/test'

In [ ]:
batch_size=32
learning_rate=0.001
epochs=25
optimizer=(tf.keras.optimizers.Adam(learning_rate=learning_rate))
dropout=0.2

In [ ]:
train_data = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

val_data = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

test_data = image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

In [ ]:
model = Sequential()

pretrained_model = DenseNet121(
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='max',
    weights='imagenet'
)

pretrained_model.trainable = False

pretrained_model = Model(
    inputs=pretrained_model.inputs,
    outputs=pretrained_model.layers[-2].output
)
model.add(pretrained_model)

model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(64, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(18, activation='softmax'))

model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy',])

In [ ]:
hist = model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=[tensorboard_callback])

In [ ]:
test_loss,test_accuracy=model.evaluate(test_data)

In [ ]:
train_accuracy=round(hist.history["accuracy"][-1], 2)
val_accuracy=round(hist.history["val_accuracy"][-1], 2)
test_accuracy=round(test_accuracy, 2)

In [ ]:
print(f'Epochs: {epochs}\nbatch Size: {batch_size}\nLearning Rate: {learning_rate}\nOptimizer: {optimizer}\nDropout: {dropout}\nTrain Accuracy: {train_accuracy} Test Accuracy: {test_accuracy} Val Accuracy: {val_accuracy}')

In [ ]:
plt.plot(hist.history['accuracy'], label=f'Training Accuracy')
plt.plot(hist.history['val_accuracy'], label=f'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc="lower right")
plt.title('Training & Validation Accuracy')
plt.show()

In [ ]:
plt.plot(hist.history['loss'], label=f'Training Loss')
plt.plot(hist.history['val_loss'], label=f'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc="upper right")
plt.title('Training & Validation Loss')
plt.show()